# Inference

We will be performing:
 - Creating an UDF that will be used for inferencing
 - Do a sample run of inference

In [18]:

from IPython.display import display, HTML, Image , Markdown
from snowflake.snowpark.session import Session
import snowflake.snowpark.types as T
import snowflake.snowpark.functions as F
import os ,configparser ,json ,logging

# Import the commonly defined utility scripts using
# dynamic path include
import sys
sys.path.append('../python/lutils')
import sflk_base as L

display(Markdown("### Initialization"))
logging.basicConfig(stream=sys.stdout, level=logging.ERROR)

# Source various helper functions
%run ./scripts/notebook_helpers.py

# Define the project home directory, this is used for locating the config.ini file
PROJECT_HOME_DIR = '../../'
config = L.get_config(PROJECT_HOME_DIR)
sp_session = L.connect_to_snowflake(PROJECT_HOME_DIR)

if(sp_session == None):
    raise Exception(f'Unable to connect to snowflake. Validate connection information ')

sp_session.use_role(f'''{config['APP_DB']['role']}''')
sp_session.use_schema(f'''{config['APP_DB']['database']}.{config['APP_DB']['schema']}''')
sp_session.use_warehouse(f'''{config['APP_DB']['snow_opt_wh']}''')

df = sp_session.sql('select current_user() ,current_role() ,current_database() ,current_schema();').to_pandas()
display(df)

### Initialization

,CURRENT_USER(),CURRENT_ROLE(),CURRENT_DATABASE(),CURRENT_SCHEMA()
0,BECKY,PUBLIC,INDSOL_DICOM_DB,PUBLIC


In [19]:
display(Markdown("Define the inference function"))

stmts = [
    f'''
    PUT file://../python/pneumonia_image_inference.py @lib_stg/scripts 
    auto_compress = false
    overwrite = true;
    '''
    ,f'''
    create or replace function infer_pneumonia(image_array_shape_0 integer ,image_array_shape_1 integer ,resized_feature varchar)
     returns integer
     language python
     runtime_version = '3.8'
     packages = ('snowflake-snowpark-python','numpy', 'pandas', 'snowflake-snowpark-python' ,'tensorflow' ,'scikit-learn')
     imports = ('@lib_stg/scripts/pneumonia_image_inference.py' 
                ,'@model_stg/pneumonia_model.joblib')
     handler = 'pneumonia_image_inference.main'
    '''
]    
    
for stmt in stmts:
    sp_session.sql(stmt).collect()

Define the inference function

In [21]:
display(Markdown("Sample inference execution"))

stmt = f''' 
    select 
        class_label ,class_label_num
        ,infer_pneumonia(image_array_shape_0 ,image_array_shape_1 ,resized_feature) as predicted_class_num
        ,IMAGE_FILE_PATH
    from images_formatted
    where IMAGE_FILE_PATH in (
        '@data_stg/NORMAL/NORMAL2-IM-1124-0001.jpeg'
        ,'@data_stg/NORMAL/NORMAL/IM-0555-0001.jpeg'
        ,'@data_stg/PNEUMONIA/person1408_bacteria_3579.jpeg'
        ,'@data_stg/NORMAL/NORMAL2-IM-0913-0001.jpeg'
        ,'@data_stg/PNEUMONIA/person1242_bacteria_3198.jpeg'
        ,'@data_stg/NORMAL/NORMAL2-IM-0587-0001.jpeg'
    )
'''
df = sp_session.sql(stmt).to_pandas()

display(df)


Sample inference execution

,CLASS_LABEL,CLASS_LABEL_NUM,PREDICTED_CLASS_NUM,IMAGE_FILE_PATH
0,NORMAL,1,1,@data_stg/NORMAL/NORMAL2-IM-0913-0001.jpeg
1,PNEUMONIA,0,0,@data_stg/PNEUMONIA/person1408_bacteria_3579.jpeg
2,PNEUMONIA,0,0,@data_stg/PNEUMONIA/person1242_bacteria_3198.jpeg
3,NORMAL,1,1,@data_stg/NORMAL/NORMAL2-IM-1124-0001.jpeg
4,NORMAL,1,1,@data_stg/NORMAL/NORMAL2-IM-0587-0001.jpeg


--- 
### Closeout

    With that we are finished this section of the demo setup

In [ ]:
sp_session.close()
print('Finished!!!')